<a href="https://colab.research.google.com/github/akksshah/AlgoWiki/blob/master/ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. I cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [2]:
def install_dependencies():
  %pip install --upgrade spacy allennlp
  import spacy
  print(spacy.__version__)
  

install_dependencies()
print("dependencies loaded")

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 7.6MB 4.2MB/s 
     |████████████████████████████████| 204kB 59.9MB/s 
     |████████████████████████████████| 256kB 61.4MB/s 
     |████████████████████████████████| 5.5MB 45.8MB/s 
     |████████████████████████████████| 133kB 62.9MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 245kB 56.3MB/s 
     |████████████████████████████████| 163kB 47.8MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 1.0MB 48.0MB/s 
  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31893 sha256=d9509bd962517645d806f07fca48d231af2b49e25e7014ecb3da070efb558251
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27feebc61dae16b84c7cdd93b8bc3d7486
  Created wheel for jsonnet: filename=js

In [3]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [0]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [5]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)
print("Load Complete")

159377B [00:00, 2617766.96B/s]
562it [00:00, 4972.06it/s]
8366B [00:00, 12364181.56B/s]
23it [00:00, 6582.22it/s]
100%|██████████| 585/585 [00:00<00:00, 41373.71it/s]

Load Complete


## Define and Train Model

In [6]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()
print("train complete")

accuracy: 0.8442, loss: 0.9055 ||: 100%|██████████| 281/281 [00:01<00:00, 175.82it/s]
accuracy: 0.7878, loss: 1.1976 ||: 100%|██████████| 12/12 [00:00<00:00, 405.66it/s]
accuracy: 0.8442, loss: 0.7297 ||: 100%|██████████| 281/281 [00:01<00:00, 252.91it/s]
accuracy: 0.7878, loss: 1.1956 ||: 100%|██████████| 12/12 [00:00<00:00, 435.37it/s]
accuracy: 0.8442, loss: 0.7153 ||: 100%|██████████| 281/281 [00:01<00:00, 253.84it/s]
accuracy: 0.7878, loss: 1.1694 ||: 100%|██████████| 12/12 [00:00<00:00, 474.04it/s]
accuracy: 0.8442, loss: 0.7067 ||: 100%|██████████| 281/281 [00:01<00:00, 249.11it/s]
accuracy: 0.7878, loss: 1.1705 ||: 100%|██████████| 12/12 [00:00<00:00, 454.91it/s]
accuracy: 0.8442, loss: 0.6987 ||: 100%|██████████| 281/281 [00:01<00:00, 255.14it/s]
accuracy: 0.7878, loss: 1.1580 ||: 100%|██████████| 12/12 [00:00<00:00, 445.39it/s]
accuracy: 0.8442, loss: 0.6896 ||: 100%|██████████| 281/281 [00:01<00:00, 251.93it/s]
accuracy: 0.7878, loss: 1.1776 ||: 100%|██████████| 12/12 [00:00

train complete


## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [0]:
def create_tuples(sequence):
  tup = []
  i = 0
  while(i < len(sequence)):
    current = sequence[i]
    if current.startswith('O'):
      i = i + 1
      # print('O tag fixed')
      continue
    else:
      tag = ''.join(current[2:])
      # print('=========',current,tag)
      j = i
      while(j < len(sequence)):
        new_current = ''.join(sequence[j][2:])
        # print('@@@@',sequence[j], j, new_current)
        # print('---',current, new_current)
        if tag.startswith(new_current) and new_current is not '':
          # print('---',current, new_current)
          j = j + 1
        else:
          # print(">>>>")
          break
      if j == i:
        tup.append([tag, i, j])
      else:
        tup.append([tag, i, j])
        i = j
      i = i + 1
  return tup

In [0]:
def get_span(sequence):
  span_list = []
  for tokens in sequence:
    # print('\n'.join(tokens))
    span_list.append(create_tuples(tokens))
  return span_list

def get_true_positive(gold_standard_span, model_output_span):
  count = 0
  for i in range(0, len(gold_standard_span)):
    gold = gold_standard_span[i]
    mod = model_output_span[i]
    for j in range(0, len(mod)):
      # print(gold[j], mod[j])
      # print(len(mod), len(gold))
      tup = mod[j]
      for item in gold:
      #   print(item , tup)
        if item[0] == tup[0] and item[1] == tup[1] and item[2] == tup[2]:
          # print('##########',item , tup)
          count = count + 1
          break
    # break
  return count

def get_recall(gold_standard_span, model_output_span):
  # print(gold_standard_span)
  true_positive = get_true_positive(gold_standard_span, model_output_span)
  # print('12345678', true_positive)
  false_postive = 0
  counts = 0
  for i in range(0, len(gold_standard_span)):
    counts = counts + len(gold_standard_span[i])
  precision = 1.0 * true_positive / counts
  return precision
  # return 0

def get_precision(gold_standard_span, model_output_span):
  true_positive = get_true_positive(gold_standard_span, model_output_span)
  false_negative = 0
  counts = 0
  for i in range(0, len(model_output_span)):
    counts = counts + len(model_output_span[i])
  recall = 1.0 * true_positive / counts
  return recall
  # return 0

def get_F1(gold_standard_span, model_output_span):
  recall = get_recall(gold_standard_span, model_output_span)
  precision = get_precision(gold_standard_span, model_output_span)
  F1 = 2.0 * recall * precision / (recall + precision)
  return F1

In [13]:
 # TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input

def get_output_of_model(tagged, k):
  output = []
  for sent in tagged:
    span_output = []
    for token in sent:
      span_output.append(token[k])
    output.append(span_output)
  # print("========", output)
  return output

def check_B_violations(previous, current):
  return 0

def check_I_violations(previous, current):
  if previous is 'O':
    # print(' O violations', previous, current)
    return 1
  previous_type = ''.join(previous[2:])
  current_type = ''.join(current[2:])
  # print("***", previous_type, current_type)
  if previous.startswith('B'):
    if not previous_type.startswith(current_type):
      # print("B tag not followed by I tag", previous, current)
      return 1
  if (previous.startswith('I')):
    if not (previous_type.startswith(current_type)):
      # print("I tag not followed by I tag", previous, current)
      return 1
  return 0

def check_O_violations(previous, current):
  return 0

def count_violations(sequence):
  count = 0
  for i in range(1, len(sequence)):
    # print(sequence[i])
    previous = sequence[i-1]
    current = sequence[i]
    if current.startswith("B"):
      count = count + check_B_violations(previous, current)
    elif current.startswith("I"):
      count = count + check_I_violations(previous, current)
    else:
      count = count + check_O_violations(previous, current)
  # print(count, "................")
  return count

def violations(tagged):
  violations_count = []
  output = get_output_of_model(tagged, 2)
  for tokens in output:
    violations_count.append(count_violations(tokens))
    # break
  return sum(violations_count)
    # break

  # return count

# TODO: return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
  model_output_span = get_span(get_output_of_model(tagged, 1))
  
  gold_standard_span = get_span(get_output_of_model(tagged, 2))
  return {'precision': get_precision(gold_standard_span, model_output_span),
          'recall': get_recall(gold_standard_span, model_output_span),
          'f1': get_F1(gold_standard_span, model_output_span)}

## You can check how many violations are made by the model output in predictor.
model_violations = violations(baseline_output)
print(f'Total violations for baseline model: {model_violations}')
# print(baseline_output)
print(f'Statistics of baseline model: {span_stats(baseline_output)}')

Total violations for baseline model: 40
Statistics of baseline model: {'precision': 0.23809523809523808, 'recall': 0.19230769230769232, 'f1': 0.21276595744680848}


## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [14]:
# k is the number of tags known to the model
k = len(vocab.get_index_to_token_vocabulary('labels'))
tags = vocab.get_index_to_token_vocabulary('labels')
transition_matrix = []
for i in range(0, k):
  row = []
  for j in range(0, k):
    # if violation not there 0 is returned but you need to have 1 so xor-ing. And if violation is there it returns 1 so by xor-ing make it 0
    row.append(np.log(1^count_violations([tags[i], tags[j]])))
  transition_matrix.append(row)
d = np.array(transition_matrix)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [15]:
# dp table 
# using top down dynamic programming approach to fill up the table and get best tagging sequence
def viterbi(probability, row, column):
  # if row is initial row then you directly need the emission probability
  if row == 0:
    return probability[row][column]
  # look up if value in cell is already filled
  if viterbi_table[row][column] != 0:
    return viterbi_table[row][column]
  maximum = np.log(0)
  backtrackptr = -1
  for col in range(len(probability[row - 1])):
    current = viterbi(probability, row - 1, col) + transition_matrix[col][column]
    if current > maximum:
      maximum = current
      backtrackptr = col
  back_ptr_choice[row][column] = backtrackptr
  viterbi_table[row][column] = maximum + probability[row][column]
  return viterbi_table[row][column]

# this is the matrix to store the tagging sequence pointer
back_ptr_choice = []
# dp look up table. Filled using the top down approach
viterbi_table = []
all_tags = vocab.get_index_to_token_vocabulary('labels')
def tag_sentence_crf(s):
  emission_prob = predictor.predict_instance(s)['tag_logits']
  # fill up dp table with 0
  for row in range(len(emission_prob)):
    viterbi_table.insert(row, [0] * len(emission_prob[row]))
    back_ptr_choice.insert(row, [0] * len(emission_prob[row]))
  maximum = np.log(0)
  # c is the tag choice for tagging seqence
  c = -1
  # fill table and get best tag sequence with 0 violations
  for col in range(len(emission_prob[len(emission_prob) - 1])):
    curr = viterbi(emission_prob, len(emission_prob) - 1, col)
    if curr > maximum:
      maximum = curr
      c = col

  crf_tagging_sequence = []
  for r in reversed(range(len(emission_prob))):
    # print(all_tags[c])
    # append the tags in sequential order
    crf_tagging_sequence.append(all_tags[c])
    c = back_ptr_choice[r][c]
  crf_tagging_sequence.reverse()
  fields = zip(s['tokens'], s['tags'], crf_tagging_sequence)
  return list(fields)

  
baseline_output_crf = [tag_sentence_crf(i) for i in validation_dataset]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [17]:
print(f'CRF model violations: {violations(baseline_output_crf)}')
print(f'Statistics of CRF model: {span_stats(baseline_output_crf)}')

CRF model violations: 0
Statistics of CRF model: {'precision': 0.5, 'recall': 0.5121951219512195, 'f1': 0.5060240963855421}
